# Image Preprocessing

### Import The ImageDataGenerator Library

In [1]:
from keras.preprocessing.image import ImageDataGenerator

### Configure ImageDataGenerator Class

In [2]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

### Apply ImageDataGenerator Functionality To Trainset And Testset

###### Loading our data and performing data agumentation

In [4]:
x_train = train_datagen.flow_from_directory('../Dataset/train', 
                                            target_size=(64, 64), batch_size=5, color_mode='grayscale', class_mode='categorical')
x_test = test_datagen.flow_from_directory('../Dataset/test', 
                                          target_size=(64, 64), batch_size=5, color_mode='grayscale', class_mode='categorical')

Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


# Model Building

### Importing The Model Building Libraries

In [5]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

### Initializing The Model

In [6]:
classifier = Sequential()

### Adding CNN Layers

In [7]:
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1) ,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())

### Adding Dense Layers

In [8]:
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=6, activation='softmax'))

In [9]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

# Configure The Learning Process

##### Compiling the model

In [10]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Model

In [11]:
classifier.fit_generator(
    generator=x_train, steps_per_epoch=len(x_train),
    epochs=20, validation_data=x_test, validation_steps=len(x_test)
)

C:\Users\Chinna\AppData\Local\Temp\ipykernel_10296\2617134232.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/20
119/119 [==============================] - 13s 104ms/step - loss: 1.3979 - accuracy: 0.4428 - val_loss: 0.9501 - val_accuracy: 0.6000
Epoch 2/20
119/119 [==============================] - 4s 33ms/step - loss: 0.7114 - accuracy: 0.7273 - val_loss: 0.5290 - val_accuracy: 0.8667
Epoch 3/20
119/119 [==============================] - 4s 32ms/step - loss: 0.5059 - accuracy: 0.8098 - val_loss: 0.6474 - val_accuracy: 0.7667
Epoch 4/20
119/119 [==============================] - 4s 32ms/step - loss: 0.3866 - accuracy: 0.8535 - val_loss: 0.3826 - val_accuracy: 0.9000
Epoch 5/20
119/119 [==============================] - 4s 33ms/step - loss: 0.3003 - accuracy: 0.8990 - val_loss: 0.3926 - val_accuracy: 0.8667
Epoch 6/20
119/119 [==============================] - 4s 34ms/step - loss: 0.2332 - accuracy: 0.9108 - val_loss: 0.2659 - val_accuracy: 0.9333
Epoch 7/20
119/119 [==============================] - 4s 36ms/step - loss: 0.1898 - accuracy: 0.9411 - val_loss: 0.2146 - val_accuracy: 0.83

# Save The Model

In [12]:
classifier.save('gesture.h5')

In [13]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as f:
    f.write(model_json)

# Test the model

### Predicting our results

In [14]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image

In [15]:
model = load_model("gesture.h5")

In [16]:
from tensorflow.keras.utils import load_img, img_to_array

img = load_img("../Dataset/test/1/0.jpg", grayscale=True, target_size=(64, 64))
x = img_to_array(img)
x = np.expand_dims(x,axis=0)
pred = model.predict(x)

pred

C:\Users\Chinna\anaconda3\lib\site-packages\keras\utils\image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


1/1 [==============================] - 0s 207ms/step


array([[0., 1., 0., 0., 0., 0.]], dtype=float32)

In [17]:
pred[0]

array([0., 1., 0., 0., 0., 0.], dtype=float32)